# 3. Modelo de Recomendación

Modelo de recomendacion por similitud de cocenos

Una vez que toda la data es consumible por la API, está lista para consumir por los departamentos de Analytics y Machine Learning, y nuestro EDA nos permite entender bien los datos a los que tenemos acceso, es hora de entrenar nuestro modelo de machine learning para armar un sistema de recomendación de películas. El EDA debería incluir gráficas interesantes para extraer datos, como por ejemplo una nube de palabras con las palabras más frecuentes en los títulos de las películas. Éste consiste en recomendar películas a los usuarios basándose en películas similares, por lo que se debe encontrar la similitud de puntuación entre esa película y el resto de películas, se ordenarán según el score de similaridad y devolverá una lista de Python con 5 valores, cada uno siendo el string del nombre de las películas con mayor puntaje, en orden descendente. Debe ser deployado como una función adicional de la API anterior y debe llamarse:

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
import re

In [ ]:
# Cargar el dataset
df = pd.read_csv("movies_dataset.csv")#--- del EDA 

In [ ]:
# Inicializa el vectorizador TF-IDF
tfidf = TfidfVectorizer(stop_words='english')

# Genera la matriz de TF-IDF para 'overview'
tfidf_matrix = tfidf.fit_transform(df['overview'])

In [ ]:
# Función para extraer y unificar la información de género, elenco, y equipo
def combine_features(row):
    return row['genres'] + " " + row['cast'] + " " + row['crew']

# Aplica la función de combinación de características
df['combined_features'] = df.apply(combine_features, axis=1)

In [ ]:
tfidf_combined = TfidfVectorizer(stop_words='english')
combined_matrix = tfidf_combined.fit_transform(df['combined_features'])

In [ ]:
# Calcula la similitud del coseno sobre la matriz TF-IDF de la combinación de características
cosine_sim = cosine_similarity(combined_matrix, combined_matrix)

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # Obtén el índice de la película que coincide con el título
    idx = df[df['title'] == title].index[0]
    
    # Obtén los puntajes de similitud de todas las películas con la película dada
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Ordena las películas según los puntajes de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Obtén los índices de las 5 películas más similares
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]
    
    # Devuelve los títulos de las 5 películas recomendadas
    return df['title'].iloc[movie_indices].tolist()